# Compute Error


## Robot setup

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.utils.extmath import cartesian
import transforms3d as t3d

import sys
sys.path.append("..")
sys.path.append("../../")

from armControl import *
from forwardKinematics import robot_config
from utils.motor_setup import maxonGearSmall
import armControl

from utils.NatNetClient2 import NatNetClient
from utils.GetJointData import data, NatNetFuncs #receiveNewFrame, receiveRigidBodyFrameList

import signal

import matplotlib.pyplot as plt

socket_ip = '192.168.0.115'
socket_port = 1126

server_ip = "192.168.0.103"
multicastAddress = "239.255.42.99"

joint_names = ['Base','RigidBody']
ids = [0, 1]

#Tracking class
print("Starting streaming client now...")
streamingClient = NatNetClient(server_ip, multicastAddress, verbose = False)
NatNet = NatNetFuncs()
streamingClient.newFrameListener = NatNet.receiveNewFrame
streamingClient.rigidBodyListListener = NatNet.receiveRigidBodyFrameList
prev_frame = 0
time.sleep(0.5)
streamingClient.run()
time.sleep(0.5)
track_data = data(joint_names,ids)
time.sleep(0.5)
track_data.parse_data(NatNet.joint_data, NatNet.frame)

#trajPlanner = trajectoryGenerator()

motors = maxonGearSmall()
motors.tcp_init(socket_ip, socket_port)

#axis_motor_indexes = np.array([-1, -1, -1, 0, 3, 2, 1]) #indexed from 0 for motors and axis
#velocity = np.ones(8)*3.14/20


Starting streaming client now...
Opening socket at ip: 192.168.0.115 using port: 1126


## Command motors w/ indexing for joints

def commandJoints(setpoint_arm, trajectory = True):
    myArm = armControl.remoteRobotArm()
    myArm.jointAngleSetpoint = setpoint_arm
    myArm.updateMotorArmMixing()
    setpoint_motor = np.zeros(8)
    setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
    if trajectory:
        motors.run_trajectory(setpoint_motor, velocity)
    else:
        motors.command_motors_radians(setpoint_motor)


## Arm Motors

In [27]:
print("Arming motors now...")
motors.arm_motors()


Arming motors now...
initializing motors to [0 0 0 0 0 0 0 0]
*** C side has an error or needs to be armed ***

Arming motors


## Manual motor zeroing

In [53]:
from importlib import reload
reload(armControl)

myArm = armControl.remoteRobotArm()

setpoint_arm = np.array([0.0,0.0,-0.,0,0.0,0,0])
myArm.commandJoints(motors, setpoint_arm)
time.sleep(0.5)


In [29]:
for i in range(8):
    motors.zero_motors_radians(i, motors.get_motors_position_radians()[i])

In [51]:
from tqdm import tqdm_notebook as tqdm

print(myArm.jointUpperLimits)
print(myArm.jointLowerLimits)

joint1_angles = np.linspace(myArm.jointLowerLimits[0]/2000,myArm.jointUpperLimits[0]/2000, num=2)
joint2_angles = np.linspace(myArm.jointLowerLimits[1]/2000,myArm.jointUpperLimits[1]/2000, num=2)
joint3_angles = np.linspace(myArm.jointLowerLimits[2]/2,myArm.jointUpperLimits[2]/2, num=2)
joint4_angles = np.linspace(myArm.jointLowerLimits[3]/2,myArm.jointUpperLimits[3]/2, num=2)
joint5_angles = np.linspace(myArm.jointLowerLimits[4]/2,myArm.jointUpperLimits[4]/2, num=2)
joint6_angles = np.linspace(myArm.jointLowerLimits[5]/1.5,myArm.jointUpperLimits[5]/2, num=2)
joint7_angles = np.zeros(1)#np.linspace(myArm.jointLowerLimits[6]/2,myArm.jointUpperLimits[6]/2, num=1)


joint_angles = [np.array(joint1_angles), np.array(joint2_angles), np.array(joint3_angles), np.array(joint4_angles), np.array(joint5_angles), np.array(joint6_angles), np.array(joint7_angles)]
PointsMatrix = cartesian(joint_angles)
num_sweeps = 3
num_static_points = 50
num_points = len(PointsMatrix[:,1])

print(np.max(PointsMatrix, axis = 0))
print(np.min(PointsMatrix, axis = 0))

Pos_Points = np.zeros(shape = (num_points, num_sweeps, 3))
Quat_Points = np.zeros(shape = (num_points, num_sweeps, 4))
static_pos_points = np.zeros(shape = (num_points, num_static_points, num_sweeps, 3))
static_quat_points = np.zeros(shape = (num_points, num_static_points, num_sweeps, 4))

pbar = tqdm(total=num_sweeps*num_points)  # set progress bar

reorder = np.arange(0,num_points)

for i in range(num_sweeps):
    np.random.shuffle(reorder)
    
    for j in range(num_points):
        
        j_reorder = reorder[j]
        
        #setpoint_base = np.array([0,0,0])
        #setpoint_arm = np.flip(np.array(PointsMatrix[j,:]),0)
        #setpoint_robot = np.append(setpoint_base, setpoint_arm)
        jointCommand = PointsMatrix[j_reorder,:].copy()
        #print('Joint angle index: {}'.format(j))
        #print('Command matrix: {}'.format(PointsMatrix))
        #print('Commanded joint angles: {}'.format(jointCommand[5]))
        
        myArm.commandJoints(motors, jointCommand, trajectory = True)
        
        time.sleep(2)
        
        track_data.parse_data(NatNet.joint_data, NatNet.frame) #updates the frame and data that is being used
        #track_data.parse_data(NatNet.joint_data, NatNet.frame) #updates the frame and data that is being used

        #add in optitrack position measurements
        
        base = track_data.bodies[0].homogenous_mat
        base_inv = track_data.bodies[0].homg_inv
        needle = track_data.bodies[1].homogenous_mat
        needle_base, needlebase_pos, needlebase_euler, _ = track_data.homg_mat_mult(base_inv,needle)
        needlebase_rot = np.array(needle_base[0:3,0:3])
        needlebase_quat = t3d.quaternions.mat2quat(needlebase_rot)
        Pos_Points[j_reorder, i, :] = needlebase_pos.copy()
        Quat_Points[j_reorder, i, :] = needlebase_quat.copy()
        
        for k in range(num_static_points):
            track_data.parse_data(NatNet.joint_data, NatNet.frame)

            base_static = track_data.bodies[0].homogenous_mat
            base_static_inv = track_data.bodies[0].homg_inv
            needle_static = track_data.bodies[1].homogenous_mat
            needle_base_static, needlebase_pos_static, _ , _ = track_data.homg_mat_mult(base_static_inv,needle_static)
            needlebase_rot_static = np.array(needle_base[0:3,0:3])
            needlebase_quat_static = t3d.quaternions.mat2quat(needlebase_rot_static)
            static_pos_points[j_reorder, k, i, :] = needlebase_pos_static.copy()
            static_quat_points[j_reorder, k, i, :] = needlebase_quat_static.copy()
        
        pbar.update(1)
        #print('Read EE position: {}\n\n'.format(needlebase_pos))
        
timestr = time.strftime("%Y%m%d-%H%M%S")
np.savez('AllJoints_random'+timestr, Pos_Points, Quat_Points, static_pos_points, static_quat_points)

[8.20e+01 4.20e+01 1.75e+00 8.72e-01 8.72e-01 8.72e-01 4.75e-02]
[-82.    -42.     -1.75   -0.872  -0.872  -0.872   0.   ]
[0.041 0.021 0.875 0.436 0.436 0.436 0.   ]
[-0.041      -0.021      -0.875      -0.436      -0.436      -0.58133333
  0.        ]


In [54]:
np.savez('AllJoints_random'+timestr, Pos_Points, Quat_Points, static_pos_points, static_quat_points)

In [55]:
Pos_Points.shape # num joint angles, repeats, xyz 
Pos_mean = Pos_Points.mean(axis=1)
pos_error = Pos_Points - Pos_mean[:, np.newaxis, :].repeat(repeats = num_sweeps, axis = 1)
pos_RMS = np.mean(np.sqrt(np.sum(pos_error * pos_error, axis = 2)))
prop_RMS_xyz = np.mean(np.abs(pos_error), axis = (0,1))
pos_max_error = np.max(np.sqrt(np.sum(pos_error * pos_error, axis = 2)))
pos_max_error_xyz = np.max(np.abs(pos_error), axis = (0,1))
print('Average error: {}'.format(pos_RMS))
print('Average error (x,y,z): {}'.format(prop_RMS_xyz))
print('Max error: {}'.format(pos_max_error))
print('Max error (x,y,z): {}'.format(pos_max_error_xyz))

cov_xyz = (pos_error.reshape(num_points*num_sweeps,3).T 
           @ pos_error.reshape(num_points*num_sweeps,3)) / (num_points*num_sweeps)

print('Position Covariance Matrix: {}'.format(cov_xyz))

#cov_pos_xy = np.divide(np.transpose(pos_error[:,:,1])*pos_error[:,:,2],num_points)
#cov_pos_xz = np.divide(np.transpose(pos_error[:,:,1])*pos_error[:,:,3],num_points)
#cov_pos_yz = np.divide(np.transpose(pos_error[:,:,2])*pos_error[:,:,3],num_points)
#print()

#calculate error in optitrack
#method 1
#static_pos_mean1 = np.mean(static_pos_points.reshape(num_points*num_static_points*num_sweeps, 3), axis = 0)
#static_pos_points1 = static_pos_points.reshape(num_points*num_static_points*num_sweeps,3)
#static_pos_error1 = static_pos_points - static_pos_mean[np.newaxis,:].repeat(repeats = num_points*num_static_points*num_sweeps, axis = 1)
#optitrack_cov_xyz1 = static_pos_error.T @ static_pos_error / (num_points*num_static_points*num_sweeps)

#print('Method 1 Optitrack Position Covariance Matrix: {}'.format(optitrack_cov_xyz1))

#method 2
static_pos_points2 = static_pos_points.reshape(num_points*num_static_points,num_sweeps,3)
static_pos_mean2 = static_pos_points2.mean(axis = 1)
static_pos_error2 = static_pos_points2 - static_pos_mean2[:, np.newaxis, :].repeat(repeats = num_sweeps, axis = 1)
optitrack_cov_xyz2 = (static_pos_error2.reshape(num_points*num_static_points*num_sweeps,3).T 
                       @ static_pos_error2.reshape(num_points*num_static_points*num_sweeps,3)) / (num_points*num_static_points*num_sweeps)

print('Method 2 Optitrack Position Covariance Matrix: {}'.format(optitrack_cov_xyz2))

Average error: 0.004171264309322354
Average error (x,y,z): [0.00297519 0.00141929 0.0015638 ]
Max error: 0.05220338723710853
Max error (x,y,z): [0.04206856 0.02538069 0.03016926]
Position Covariance Matrix: [[ 4.28982002e-05 -3.38516977e-06  9.19494337e-06]
 [-3.38516977e-06  8.57741544e-06 -5.79485181e-06]
 [ 9.19494337e-06 -5.79485181e-06  1.04357022e-05]]
Method 2 Optitrack Position Covariance Matrix: [[ 4.29207103e-05 -3.47558253e-06  9.25029236e-06]
 [-3.47558253e-06  8.70145958e-06 -5.90482600e-06]
 [ 9.25029236e-06 -5.90482600e-06  1.05153311e-05]]


In [64]:
from utils.quaternion_functions import quat_average
import transforms3d as t3d
import numpy as np

#https://www.andre-gaschler.com/rotationconverter/
#q1 = [0, 1, 0, 0] # 180 degree rotation around axis 0
#q2 = [0.7071068, 0.7071068, 0, 0] # 90 degree rotation around axis 0
#avg = quat_average(np.array([q1,q2]),np.ones(2)/2)
#print('axis angle of q1: {}'.format(t3d.quaternions.quat2axangle(q1)))
#print('axis angle of q2: {}'.format(t3d.quaternions.quat2axangle(q2)))
#print('average: {}'.format(t3d.quaternions.quat2axangle(avg)))


#Pos_Points = np.zeros(shape = (num_points, num_sweeps, 3))
#Quat_Points = np.zeros(shape = (num_points, num_sweeps, 4))

avg_quaternions = np.zeros((num_points,4))
avg_quaternions_inverse = np.zeros((num_points,4))
quaternion_error = np.zeros((num_points, num_sweeps , 4))
euler_angle_error = np.zeros((num_points, num_sweeps, 3))

for i in range(num_points):
    avg_quaternions[i,:] = quat_average(Quat_Points[i,:,:], np.ones(num_sweeps)/num_sweeps)
    avg_quaternions_inverse[i,:] = t3d.quaternions.qinverse(avg_quaternions[i,:])

for i in range(num_points):
    for j in range(num_sweeps):
        quaternion_error[i,j,:] = t3d.quaternions.qmult(Quat_Points[i,j,:],avg_quaternions_inverse[i,:])
        euler_angle_error[i,j,:] = t3d.euler.quat2euler(quaternion_error[i,j,:])

avg_quaternion_bias = quat_average(quaternion_error.reshape((num_sweeps*num_points,4)), 
                                    np.ones(num_sweeps*num_points)/(num_sweeps*num_points))

rms_euler_angle = np.mean(np.sqrt(np.sum(euler_angle_error*euler_angle_error, axis = 2)))*180/np.pi
euler_angle_error_mean = np.mean(np.sqrt(euler_angle_error*euler_angle_error), axis=(0,1))*180/np.pi

print('euler angle mean bias: {}'
      .format(t3d.euler.quat2euler(avg_quaternion_bias)))
print('rms euler angle error (degrees): {}'
      .format(rms_euler_angle))

print('euler angle error mean (degrees, xyz): {}'.format(euler_angle_error_mean))

cov_rpy = euler_angle_error.reshape(num_points*num_sweeps,3).T @ euler_angle_error.reshape(num_points*num_sweeps,3) / (num_points*num_sweeps)

print('Euler Angle Covariance Matrix: {}'.format(cov_rpy))

#calculate error in optitrack
avg_static_quaternions = np.zeros((num_points*num_static_points,4))
avg_static_quaternions_inverse = np.zeros((num_points*num_static_points,4))
quaternion_static_error = np.zeros((num_points*num_static_points, num_sweeps , 4))
euler_angle_static_error = np.zeros((num_points*num_static_points, num_sweeps, 3))
static_quat_points = static_quat_points.reshape(num_points*num_static_points, num_sweeps, 4)
for i in range(num_points*num_static_points):
    avg_static_quaternions[i,:] = quat_average(static_quat_points[i,:,:], np.ones(num_sweeps)/num_sweeps)
    avg_static_quaternions_inverse[i,:] = t3d.quaternions.qinverse(avg_static_quaternions[i,:])

for i in range(num_points*num_static_points):
    for j in range(num_sweeps):
        quaternion_static_error[i,j,:] = t3d.quaternions.qmult(static_quat_points[i,j,:],avg_static_quaternions_inverse[i,:])
        euler_angle_static_error[i,j,:] = t3d.euler.quat2euler(quaternion_static_error[i,j,:])

optitrack_cov_rpy = euler_angle_static_error.reshape(num_points*num_static_points*num_sweeps,3).T @ euler_angle_static_error.reshape(num_points*num_static_points*num_sweeps,3) / (num_points*num_sweeps)
    
print('Optitrack Euler Angle Covariance Matrix: {}'.format(optitrack_cov_rpy))


#THIS IS BIAS NOT ERROR!!!

#avg_inv = t3d.quaternions.qinverse(avg)    
#q1_error = t3d.quaternions.qmult(q1,avg_inv)
#q2_error = t3d.quaternions.qmult(q2,avg_inv)
#print('axis angle difference from mean of q1: {}'
#      .format(t3d.quaternions.quat2axangle(q1_error)))
#print('axis angle difference from mean of q2: {}'
#      .format(t3d.quaternions.quat2axangle(q2_error)))


euler angle mean bias: (-1.6729452664367343e-08, -8.201835638919677e-09, -1.965658902009777e-08)
rms euler angle error (degrees): 2.673392019933526
euler angle error mean (degrees, xyz): [1.34786739 1.51044091 1.08831622]
Euler Angle Covariance Matrix: [[ 0.00355553 -0.00136268 -0.0006644 ]
 [-0.00136268  0.00200973  0.00031462]
 [-0.0006644   0.00031462  0.00208604]]
Optitrack Euler Angle Covariance Matrix: [[ 0.17777675 -0.06813402 -0.03321994]
 [-0.06813402  0.10048633  0.01573108]
 [-0.03321994  0.01573108  0.10430204]]


In [61]:
euler_angle_static_error.shape

(3200, 3, 3)

In [62]:
static_pos_error2.shape

(3200, 3, 3)

In [63]:
num_points*num_static_points*num_sweeps,3

(9600, 3)

#Compute Avg of Positions (x,y,z)
Pos_Avg = np.zeros(shape = (1, 3, num_sweeps))
for i in range(num_sweeps):
    Pos_Avg(1,:,i) = np.mean(Pos_Points, axis = 0)
Pos_Error = np.zeros(shape = (num_points, 3, num_sweeps))
sweepNum = np.linspace(1,num_sweeps,num = num_sweeps)

#Compute Error of Positions (x,y,z)
for j in range(num_sweeps):
    for k in range(num_points):
        Pos_Error = Pos_Points[k,:,j] - Pos_Avg[1,:,j]
    print("Sweep {} Position Error: {}".format(sweepNum[j], Pos_Error[:,:,j]))



## Cleanup

In [17]:
setpoints = np.zeros(8)
velocity = np.ones(8)*3.14/5
motors.run_trajectory(setpoints, velocity)
time.sleep(1)
motors.tcp_close()

In [ ]:
motors.tcp_close()